<a href="https://colab.research.google.com/github/ShubhamW248/LLM-Practice/blob/main/Tokens_Embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introduction to Tokens and Embeddings

In this notebook, we delve into two key concepts essential for understanding and working with language models: **Tokens** and **Embeddings**. These concepts form the foundation for efficient text processing and enable applications like recommendation systems and text similarity tasks.

---

## Tokens

Tokens are the fundamental units of text that a language model processes. They can be words, subwords, or even characters depending on the tokenization method used. We will explore the following aspects:

1. **Tokenization Method**  
   The technique used to split the input text into tokens (e.g., Byte Pair Encoding, WordPiece, or SentencePiece).  

2. **Vocabulary Size**  
   The number of unique tokens the model can recognize and work with. Larger vocabularies allow for more nuanced representations but come with increased computational cost.

3. **Special Tokens**  
   Tokens reserved for specific purposes like indicating the start (`<s>`), end (`</s>`), or padding of a sequence, as well as separating sentences or marking unknown words (`<unk>`).

---

## Embeddings

Embeddings are dense vector representations of text that encode semantic and syntactic information. These numerical representations are critical for tasks requiring the comparison of text, such as similarity, clustering, and recommendation systems.

### Key Topics

1. **Sentence and Text Embeddings**  
   Learn how embeddings capture the meaning of individual sentences or entire documents. Models like BERT, Sentence-BERT, and OpenAI's embeddings specialize in generating high-quality embeddings for downstream tasks.

2. **Recommendation Systems Using Embeddings**  
   Explore how embeddings can be used in building recommendation systems by comparing user preferences and item descriptions in a shared vector space.

---

Follow along as we explore these concepts in depth and implement practical applications using tokens and embeddings!


In [1]:
!pip install transformers>=4.41.2 sentence-transformers>=3.0.1 gensim>=4.3.2 scikit-learn>=1.5.0 accelerate>=0.31.0


In [2]:
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load model and tokenizer
model = AutoModelForCausalLM.from_pretrained(
    "microsoft/Phi-3-mini-4k-instruct",
    device_map="cuda",
    torch_dtype="auto",
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained("microsoft/Phi-3-mini-4k-instruct")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py:   0%|          | 0.00/11.2k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py:   0%|          | 0.00/73.2k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.94M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [3]:
prompt = "Write a sweet message to our doctors.<|assistant|>"

# Tokenize the input prompt
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

# Generate the text
generation_output = model.generate(
  input_ids=input_ids,
  max_new_tokens=20
)

# Print the output
print(tokenizer.decode(generation_output[0]))


The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
`get_max_cache()` is deprecated for all Cache classes. Use `get_max_cache_shape()` instead. Calling `get_max_cache()` will raise error from v4.48


Write a sweet message to our doctors.<|assistant|> Dear Doctors,

We want to take a moment to express our deepest gratitude


In [4]:
print(input_ids)


tensor([[14350,   263, 14225,  2643,   304,  1749,   437, 14359, 29889, 32001]],
       device='cuda:0')


In [5]:
generation_output


tensor([[14350,   263, 14225,  2643,   304,  1749,   437, 14359, 29889, 32001,
           360,   799,  1938, 14359, 29892,    13,    13,  4806,   864,   304,
          2125,   263,  3256,   304,  4653,  1749,  6483,   342, 20715,  4279]],
       device='cuda:0')

In [10]:
print(tokenizer.decode(14350))
print(tokenizer.decode(263))
print(tokenizer.decode(14225))
print(tokenizer.decode(2643))


Write
a
sweet
message


#Comparing Pre-Trained Tokenizers

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer

colors_list = [
    '102;194;165', '252;141;98', '141;160;203',
    '231;138;195', '166;216;84', '255;217;47'
]

def show_tokens(sentence, tokenizer_name):
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_name)
    token_ids = tokenizer(sentence).input_ids
    for idx, t in enumerate(token_ids):
        print(
            f'\x1b[0;30;48;2;{colors_list[idx % len(colors_list)]}m' +
            tokenizer.decode(t) +
            '\x1b[0m',
            end=' '
        )


In the following text, we try to compare how different tokenizers process
tokens

Capitalization

Special characters

Mathematical Expressions

Code snippets

End Of Line



In [12]:
text = """
English and CAPITALIZATION
🎵 鸟
show_tokens False None elif == >= else: two tabs:"    " Three tabs: "       "
12.0*50=600
"""


In [14]:
show_tokens(text, "bert-base-uncased")


[CLS] english and capital ##ization [UNK] [UNK] show _ token ##s false none eli ##f = = > = else : two tab ##s : " " three tab ##s : " " 12 . 0 * 50 = 600 [SEP] 

In [16]:
show_tokens(text, "bert-base-cased")


[CLS] English and CA ##PI ##TA ##L ##I ##Z ##AT ##ION [UNK] [UNK] show _ token ##s F ##als ##e None el ##if = = > = else : two ta ##bs : " " Three ta ##bs : " " 12 . 0 * 50 = 600 [SEP] 

In [18]:
show_tokens(text, "gpt2")



 English  and  CAP ITAL IZ ATION 
 � � �  � � � 
 show _ t ok ens  False  None  el if  ==  >=  else :  two  tabs :"        "  Three  tabs :  "              " 
 12 . 0 * 50 = 600 
 

In [20]:
show_tokens(text, "google/flan-t5-small")


English and CA PI TAL IZ ATION  <unk>  <unk> show _ to ken s Fal s e None  e l if = = > = else : two tab s : " " Three tab s : " " 12. 0 * 50 = 600  </s> 

In [22]:
show_tokens(text, "Xenova/gpt-4")



 English  and  CAPITAL IZATION 
 � � �  � � � 
 show _tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         "
 12 . 0 * 50 = 600 
 

In [24]:
show_tokens(text, "bigcode/starcoder2-15b")



 English  and  CAPITAL IZATION 
 � � �   � � 
 show _ tokens  False  None  elif  ==  >=  else :  two  tabs :"      "  Three  tabs :  "         " 
 1 2 . 0 * 5 0 = 6 0 0 
 

#Contextualized Word Embeddings From a Language Model (Like BERT)


In [25]:
from transformers import AutoModel, AutoTokenizer

# Load a tokenizer
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-base")

# Load a language model
model = AutoModel.from_pretrained("microsoft/deberta-v3-xsmall")

# Tokenize the sentence
tokens = tokenizer('Hello world', return_tensors='pt')

# Process the tokens
output = model(**tokens)[0]


tokenizer_config.json:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/474 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/578 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/241M [00:00<?, ?B/s]

In [26]:
output.shape


torch.Size([1, 4, 384])

In [27]:
for token in tokens['input_ids'][0]:
    print(tokenizer.decode(token))


[CLS]
Hello
 world
[SEP]


In [28]:
output


tensor([[[-3.4816,  0.0861, -0.1819,  ..., -0.0612, -0.3911,  0.3017],
         [ 0.1898,  0.3208, -0.2315,  ...,  0.3714,  0.2478,  0.8048],
         [ 0.2071,  0.5036, -0.0485,  ...,  1.2175, -0.2292,  0.8582],
         [-3.4278,  0.0645, -0.1427,  ...,  0.0658, -0.4367,  0.3834]]],
       grad_fn=<NativeLayerNormBackward0>)

#Text Embeddings (For Sentences and Whole Documents)


In [29]:
from sentence_transformers import SentenceTransformer

# Load model
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# Convert text to text embeddings
vector = model.encode("Maximus the Greatest!")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [30]:
vector.shape


(768,)

In [33]:
import gensim.downloader as api

model = api.load("glove-wiki-gigaword-50")


[==================================================] 100.0% 66.0/66.0MB downloaded


In [36]:
model.most_similar([model['guru']], topn=11)


[('guru', 1.0),
 ('nanak', 0.8144606351852417),
 ('disciple', 0.7659479379653931),
 ('gobind', 0.7216941714286804),
 ('swami', 0.6984915137290955),
 ('gorakhnath', 0.6831441521644592),
 ('tegh', 0.6735220551490784),
 ('gurus', 0.662555456161499),
 ('ravidass', 0.6575841903686523),
 ('shri', 0.6549018025398254),
 ('dev', 0.6504449844360352)]

#Recommending songs by embeddings


In [37]:
import pandas as pd
from urllib import request

# Get the playlist dataset file
data = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/train.txt')

# Parse the playlist dataset file. Skip the first two lines as
# they only contain metadata
lines = data.read().decode("utf-8").split('\n')[2:]

# Remove playlists with only one song
playlists = [s.rstrip().split() for s in lines if len(s.split()) > 1]

# Load song metadata
songs_file = request.urlopen('https://storage.googleapis.com/maps-premium/dataset/yes_complete/song_hash.txt')
songs_file = songs_file.read().decode("utf-8").split('\n')
songs = [s.rstrip().split('\t') for s in songs_file]
songs_df = pd.DataFrame(data=songs, columns = ['id', 'title', 'artist'])
songs_df = songs_df.set_index('id')

In [38]:
print( 'Playlist #1:\n ', playlists[0], '\n')
print( 'Playlist #2:\n ', playlists[1])


Playlist #1:
  ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '2', '42', '43', '44', '45', '46', '47', '48', '20', '49', '8', '50', '51', '52', '53', '54', '55', '56', '57', '25', '58', '59', '60', '61', '62', '3', '63', '64', '65', '66', '46', '47', '67', '2', '48', '68', '69', '70', '57', '50', '71', '72', '53', '73', '25', '74', '59', '20', '46', '75', '76', '77', '59', '20', '43'] 

Playlist #2:
  ['78', '79', '80', '3', '62', '81', '14', '82', '48', '83', '84', '17', '85', '86', '87', '88', '74', '89', '90', '91', '4', '73', '62', '92', '17', '53', '59', '93', '94', '51', '50', '27', '95', '48', '96', '97', '98', '99', '100', '57', '101', '102', '25', '103', '3', '104', '105', '106', '107', '47', '108', '109', '110', '111', '112', '113', '25', '63', '62', '114', '115', '84', '116', '117',

In [39]:
from gensim.models import Word2Vec

# Train our Word2Vec model
model = Word2Vec(
    playlists, vector_size=32, window=20, negative=50, min_count=1, workers=4
)


In [40]:
print(songs_df.iloc[2172])


title     Fade To Black
artist        Metallica
Name: 2172 , dtype: object


In [41]:
import numpy as np

def print_recommendations(song_id):
    similar_songs = np.array(
        model.wv.most_similar(positive=str(song_id),topn=5)
    )[:,0]
    return  songs_df.iloc[similar_songs]

# Extract recommendations
print_recommendations(2172)


,title,artist
id,,
2976,I Don't Know,Ozzy Osbourne
3126,Heavy Metal,Sammy Hagar
2902,Jailbreak,Thin Lizzy
3136,Mean Street,Van Halen
6626,Blackout,Scorpions


In [42]:
print_recommendations(842)


,title,artist
id,,
413,If I Ruled The World (Imagine That) (w\/ Laury...,Nas
6741,Love In This Club (w\/ Young Jeezy),Usher
1418,Tick Tock,Kesha
890,Knock You Down (w\/ Ne-Yo & Kanye West),Keri Hilson
5668,How We Do (w\/ 50 Cent),The Game


Looks Correct!